In [28]:
import pandas as pd
import re


In [29]:
df_reports = pd.read_csv('a:/bloeding-met-patientenlijst-gedetailleerd/verslagen-abb-corrected.csv')
df_reports['verslagen_report_start_date'] = pd.to_datetime(df_reports['verslagen_report_start_date'], errors='coerce')

df_reports.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,abbreviations_corrected
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",87,"Maag-, Darm- en Leverziekten",2020-11-26 15:06:00,"meneer a.j. dingemans, huisarts\r\n[streetname..."
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,87,"Maag-, Darm- en Leverziekten",2020-11-26 09:53:00,samenvatting: \nrectaal bloedverlies onder beg...
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,87,"Maag-, Darm- en Leverziekten",2020-11-25 14:13:00,coloscopie\r\n\r\nbetreft\r\nmevrouw [initials...
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,87,"Maag-, Darm- en Leverziekten",2020-11-25 13:48:00,gastroscopie\r\n\r\nbetreft\r\nmevrouw [initia...
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,87,"Maag-, Darm- en Leverziekten",2020-11-25 08:47:00,samenvatting: \nrectaal bloedverlies ; eenmali...


In [30]:
df_measures = df_reports[['pseudo_id', 'verslagen_report_start_date', 'verslagen_report_specialism', 'verslagen_report_content']]
df_measures.head()

,pseudo_id,verslagen_report_start_date,verslagen_report_specialism,verslagen_report_content
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 15:06:00,"Maag-, Darm- en Leverziekten","Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ..."
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 09:53:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies obv diver...
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 14:13:00,"Maag-, Darm- en Leverziekten",COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 13:48:00,"Maag-, Darm- en Leverziekten",GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 08:47:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies ; eenmali...


In [31]:
agib_terms = [
    r"\bacute gi\s*bloeding\b",                       # explicit AGIB phrasing
    r"\bacute maagdarmbloeding\b",                    # Dutch GI bleed
    r"\bacute gastro[-\s]?intestinale bloeding\b",    # formal wording
    r"\b agib \b"                                       # abbreviation
]


agib_terms_MDL = agib_terms + [
    r"\bacute bloeding\b",               # can mean GI in MDL context
    # r"\bmassaal bloedverlies\b",         # could suggest AGIB if noted by MDL
    # r"\bshock door bloeding\b",          # common phrasing in acute cases
    # r"\bspoed scopie\b",                 # urgent scope suggests AGIB context
    # r"\bhematemesis\b",                  # AGIB if acute presentation
    # r"\bmelaena met hypotensie\b"        # combo is a classic AGIB signal
]


gib_terms = [
    r'\bmel[ae]na\b',                        # black tarry stool
    r'\bhematemesis\b',                      # vomiting blood
    r'\bhematochez[iy]e\b',                  # fresh blood per rectum
    r'\brectaal bloed\b',                    # rectal blood
    r'\brectaal verlies\b',                  # rectal blood loss
    r'\bzwart(e)? ontlasting\b',             # black stool
    r'\bbloed in ontlasting\b',              # blood in stool
    r'\bdivertikelbloeding\b',               # bleeding from diverticula
    r'\b(bloed)?stolsels\b',                 # clots
    r'\bgastro[-\s]?intestinale bloeding\b', # literal GI bleeding
    r'\bhemorragie\b'                        # hemorrhage (less common)
]

gib_terms_MDL = gib_terms + [
    r'\bbloedverlies\b',               # general blood loss
    r'\bvers bloed per anum\b',        # fresh blood per anus
    r'\bbloedende divertikels?\b',     # bleeding diverticula
    r'\bmaagdarmbloeding\b',           # Dutch variant of GI bleeding
    r'\binvloed van antistolling\b',   # bleeding risk via anticoagulation
    r'\bhemoglobinedaling\b',          # drop in Hb, in GI context implies GIB
    r'\bbloedende ulcus\b',            # bleeding ulcer
    r'\bbloedend varices?\b'           # variceal bleeding
]


scopie_terms = [
    r'\bgastroscopie\b', r'\bendoscopie\b', r'\bcoloscopie\b', r'\bercp\b',
    r'\bvideo[-\s]?endoscopie\b', r'\bendoscopisch\b',
    r'\bspoed[-\s]?gastroscopie\b', r'\burgente coloscopie\b'
]


treatment_terms = [
    r'\bpantoprazol\b', 
    r'\bperfusor\b', 
    r'\binfuus\b', 
    r'\btransfusie\b',
    r'\bacenocoumarol\b', 
    r'\bbloeddruk (laag|dalend)\b', 
    r'\bhypotens(ie|ief)\b',
    r'\bvitamine k\b',              # reversal agent for coumarin
    r'\bprotonpompremmer\b',        # class of GI bleed meds
    r'\biv vocht\b',                # fluid support in shock
    r'\bspoedopname\b'              # linked to acute presentation
]




In [32]:
negation_triggers = r"\b(geen|niet|zonder|uitsluiten|ontkent|ontkend|ontkennen|negatief voor|geen aanwijzing voor|kan uitgesloten worden|geen tekenen van|niet passend bij|afwezig|geen bewijs voor|uitsluitend)\b"


In [33]:
def match_terms(text, patterns):
    text = text.lower()
    return any(re.search(pattern, text) for pattern in patterns)
    # text = text.lower()
    # return any(pattern.search(text) for pattern in patterns)


def match_term_per_specialism(df, text_column, terms, terms_MDL,  new_column_name='new_column', specialism_column='verslagen_report_specialism', specialism_value='Maag-, Darm- en Leverziekten'):
    """
    Match terms in a DataFrame column and create a new column with the results.
    """
    # Create a new column with the results
    # df[new_column_name] = df[text_column].apply(lambda x: match_terms(x, terms_MDL) if df[specialism_column][x] == specialism_value else match_terms(x, terms))
    df[new_column_name] = df.apply(
    lambda row: match_terms(row[text_column], terms_MDL) if row[specialism_column] == specialism_value else match_terms(row[text_column], terms),
    axis=1
)
    # # Apply negation check
    # negation = df[text_column].str.contains(negation_triggers, case=False)
    # df.loc[negation, new_column_name] = False
    



In [34]:
match_term_per_specialism(df_measures, 'verslagen_report_content', agib_terms, agib_terms_MDL, new_column_name='has_agib', specialism_column='verslagen_report_specialism', specialism_value='Maag-, Darm- en Leverziekten')
match_term_per_specialism(df_measures, 'verslagen_report_content', gib_terms, gib_terms_MDL, new_column_name='has_gib', specialism_column='verslagen_report_specialism', specialism_value='Maag-, Darm- en Leverziekten')

C:\Users\Y.vanMegen\AppData\Local\Temp\ipykernel_25680\256674290.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name] = df.apply(
C:\Users\Y.vanMegen\AppData\Local\Temp\ipykernel_25680\256674290.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name] = df.apply(


In [35]:
print(df_measures['has_agib'].value_counts())
print(len(df_measures[df_measures['has_agib']]['pseudo_id'].unique()))


False    11067
True        23
Name: has_agib, dtype: int64
3


In [36]:
print(df_measures['has_gib'].value_counts())
print(len(df_measures[df_measures['has_gib']]['pseudo_id'].unique()))

False    8307
True     2783
Name: has_gib, dtype: int64
102


In [37]:
df_measures['has_agib2'] = df_measures['verslagen_report_content'].apply(lambda x: match_terms(x, agib_terms))
df_measures['has_gib_symptoms'] = df_measures['verslagen_report_content'].apply(lambda x: match_terms(x, gib_terms))
df_measures['has_scopie_reference'] = df_measures['verslagen_report_content'].apply(lambda x: match_terms(x, scopie_terms))
df_measures['has_gib_treatment'] = df_measures['verslagen_report_content'].apply(lambda x: match_terms(x, treatment_terms))

df_measures

,pseudo_id,verslagen_report_start_date,verslagen_report_specialism,verslagen_report_content,has_agib,has_gib,has_agib2,has_gib_symptoms,has_scopie_reference,has_gib_treatment
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 15:06:00,"Maag-, Darm- en Leverziekten","Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",False,True,False,True,True,True
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 09:53:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies obv diver...,False,True,False,True,True,True
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 14:13:00,"Maag-, Darm- en Leverziekten",COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,False,True,False,True,True,True
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 13:48:00,"Maag-, Darm- en Leverziekten",GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,False,True,False,True,True,True
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 08:47:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies ; eenmali...,False,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...
11085,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2015-03-20 08:13:00,Interne Geneeskunde,Samenvatting: \n1e consult\r\n-Type 1e consult...,False,False,False,False,True,False
11086,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2015-01-14 15:39:00,Interne Geneeskunde,Samenvatting: \nDecursus\r\n-Type decursus: De...,False,False,False,False,False,False
11087,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2014-12-21 09:31:00,Spoedeisende Hulp,Samenvatting: \nVerpleegkundige verslaglegging...,False,False,False,False,False,False
11088,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2010-11-10 21:03:00,Spoedeisende Hulp,Samenvatting: \nMedisch Dossier\r\n[ Vk Sputov...,False,False,False,False,False,False


In [38]:
df_measures['has_agib2'].value_counts()

False    11090
Name: has_agib2, dtype: int64

In [39]:
df_measures['has_gib_symptoms'].value_counts()

False    9140
True     1950
Name: has_gib_symptoms, dtype: int64

In [40]:
print('has_gib_symptoms: \n', df_measures['has_gib_symptoms'].value_counts())
print('has_scopie_reference: \n', df_measures['has_scopie_reference'].value_counts())
print('has_gib_treatment: \n', df_measures['has_gib_treatment'].value_counts())

has_gib_symptoms: 
 False    9140
True     1950
Name: has_gib_symptoms, dtype: int64
has_scopie_reference: 
 False    7822
True     3268
Name: has_scopie_reference, dtype: int64
has_gib_treatment: 
 False    7810
True     3280
Name: has_gib_treatment, dtype: int64


In [41]:
def extract_context(text, pattern, window=5):
    text = text.lower()
    matches = []
    for match in pattern.finditer(text):
        start, end = match.start(), match.end()
        before = text[:start]
        after = text[end:]
        before_words = re.findall(r'\w+', before)[-window:]
        after_words = re.findall(r'\w+', after)[:window]
        context = ' '.join(before_words + [match.group()] + after_words)
        matches.append(context)
    return matches[0] if matches else None

def extract_match(text, pattern):
    match = pattern.search(text.lower())
    return match.group() if match else None

def check_findings_pos(df, text_column, terms, new_column_name='has_findings'):
    pattern_str = "|".join([fr"\b{t}\b" if " " not in t and "[- ]" not in t else t for t in terms])
    pattern = re.compile(pattern_str, re.IGNORECASE)

    df[new_column_name + '_mention' ] = df[text_column].apply(lambda x: bool(pattern.search(str(x))))

    df[new_column_name + '_match'] = df[text_column].apply(
    lambda x: extract_match(str(x), pattern)    )

    df[new_column_name + '_context'] = df['verslagen_report_content'].apply(
    lambda x: extract_context(str(x), pattern)    )

    



In [42]:
check_findings_pos(df_measures, 'verslagen_report_content', agib_terms, new_column_name='agib')
check_findings_pos(df_measures, 'verslagen_report_content', gib_terms, new_column_name='gib')

In [43]:
df_measures.head()

,pseudo_id,verslagen_report_start_date,verslagen_report_specialism,verslagen_report_content,has_agib,has_gib,has_agib2,has_gib_symptoms,has_scopie_reference,has_gib_treatment,agib_mention,agib_match,agib_context,gib_mention,gib_match,gib_context
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 15:06:00,"Maag-, Darm- en Leverziekten","Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",False,True,False,True,True,True,False,None,None,True,melena,spontane conversie naar sinusritme 2016 melena...
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 09:53:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies obv diver...,False,True,False,True,True,True,False,None,None,True,divertikelbloeding,samenvatting rectaal bloedverlies obv divertik...
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 14:13:00,"Maag-, Darm- en Leverziekten",COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,False,True,False,True,True,True,False,None,None,True,melena,t kamphuis indicatie bloedverlies rectaal mele...
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 13:48:00,"Maag-, Darm- en Leverziekten",GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,False,True,False,True,True,True,False,None,None,True,melena,verwijzer j t kamphuis indicatie melena en blo...
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 08:47:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies ; eenmali...,False,True,False,True,True,True,False,None,None,True,divertikelbloeding,stabiel inr 2 8 wd divertikelbloeding dd toch ...


In [44]:
agib_terms = [
    r"\bacute gi\s*bloeding\b",                       # explicit AGIB phrasing
    r"\bacute maagdarmbloeding\b",                    # Dutch GI bleed
    r"\bacute gastro[-\s]?intestinale bloeding\b",    # formal wording
    r"\b agib \b"                                       # abbreviation
]


agib_terms_MDL = agib_terms + [
    r"\bacute bloeding\b",               # can mean GI in MDL context
    r"\bmassaal bloedverlies\b",         # could suggest AGIB if noted by MDL
    r"\bshock door bloeding\b",          # common phrasing in acute cases
    r"\bspoed scopie\b",                 # urgent scope suggests AGIB context
    r"\bhematemesis\b",                  # AGIB if acute presentation
    r"\bmelaena met hypotensie\b"        # combo is a classic AGIB signal
]


gib_terms = [
    r'\bmel[ae]na\b',                        # black tarry stool
    r'\bhematemesis\b',                      # vomiting blood
    r'\bhematochez[iy]e\b',                  # fresh blood per rectum
    r'\brectaal bloed\b',                    # rectal blood
    r'\brectaal verlies\b',                  # rectal blood loss
    r'\bzwart(e)? ontlasting\b',             # black stool
    r'\bbloed in ontlasting\b',              # blood in stool
    r'\bdivertikelbloeding\b',               # bleeding from diverticula
    r'\b(bloed)?stolsels\b',                 # clots
    r'\bgastro[-\s]?intestinale bloeding\b', # literal GI bleeding
    r'\bhemorragie\b'                        # hemorrhage (less common)
]

gib_terms_MDL = gib_terms + [
    r'\bbloedverlies\b',               # general blood loss
    r'\bvers bloed per anum\b',        # fresh blood per anus
    r'\bbloedende divertikels?\b',     # bleeding diverticula
    r'\bmaagdarmbloeding\b',           # Dutch variant of GI bleeding
    r'\binvloed van antistolling\b',   # bleeding risk via anticoagulation
    r'\bhemoglobinedaling\b',          # drop in Hb, in GI context implies GIB
    r'\bbloedende ulcus\b',            # bleeding ulcer
    r'\bbloedend varices?\b'           # variceal bleeding
]

In [45]:
def prepare_bleeding_target_pattern(term_list):
    # Remove surrounding \b from each term (if present)
    cleaned_terms = [re.sub(r'^\\b|\\b$', '', t) for t in term_list]
    # Join terms with OR and wrap with \b(...) \b
    return r"\b(" + "|".join(cleaned_terms) + r")\b"


In [46]:
# Combine term lists as needed
combined_terms = agib_terms + gib_terms
combined_terms_MDL = agib_terms_MDL + gib_terms_MDL

# Create regex patterns
bleeding_targets = prepare_bleeding_target_pattern(combined_terms)
bleeding_targets_MDL = prepare_bleeding_target_pattern(combined_terms_MDL)

print("General:", bleeding_targets)
print("MDL:", bleeding_targets_MDL)


General: \b(acute gi\s*bloeding|acute maagdarmbloeding|acute gastro[-\s]?intestinale bloeding| agib |mel[ae]na|hematemesis|hematochez[iy]e|rectaal bloed|rectaal verlies|zwart(e)? ontlasting|bloed in ontlasting|divertikelbloeding|(bloed)?stolsels|gastro[-\s]?intestinale bloeding|hemorragie)\b
MDL: \b(acute gi\s*bloeding|acute maagdarmbloeding|acute gastro[-\s]?intestinale bloeding| agib |acute bloeding|massaal bloedverlies|shock door bloeding|spoed scopie|hematemesis|melaena met hypotensie|mel[ae]na|hematemesis|hematochez[iy]e|rectaal bloed|rectaal verlies|zwart(e)? ontlasting|bloed in ontlasting|divertikelbloeding|(bloed)?stolsels|gastro[-\s]?intestinale bloeding|hemorragie|bloedverlies|vers bloed per anum|bloedende divertikels?|maagdarmbloeding|invloed van antistolling|hemoglobinedaling|bloedende ulcus|bloedend varices?)\b


In [47]:
# # Apply regex
# # df_measures['agib_mention'] = df_measures['verslagen_report_content'].apply(lambda x: bool(agib_pattern.search(str(x))))

# # Optionally exclude negated matches
# negation_triggers = r"\b(geen|niet|zonder|uitsluiten|ontkent|ontkend|ontkennen|negatief voor|geen aanwijzing voor|kan uitgesloten worden|geen tekenen van)\b"
# # bleeding_targets = r"\b(bloeding(en)?|bloeding uit het maag[- ]darmkanaal|maagdarmbloeding|gastro[- ]intestinale bloeding|hematemesis|melena|rectaal bloedverlies|vers bloed per anum|bloed(braken)?)\b"
# bleeding_targets = "\b(acute gi\s*bloeding|acute bloeding|acute maagdarmbloeding|acute gastro[-\s]?intestinale bloeding| agib )\b"


# # Match pattern where negation is near a bleeding mention (within ~8 words)
# negation_pattern = re.compile(
#     fr"({negation_triggers})(\W+\w+){{0,20}}?({bleeding_targets})|({bleeding_targets})(\W+\w+){{0,20}}?({negation_triggers})",
#     flags=re.IGNORECASE
# )

# negation_pattern = re.compile(r"\b(geen|niet|zonder)\b.{0,20}?\b(bloed(braken)?|hematemesis|melena|maagdarmbloeding|gastro[- ]intestinale bloeding|rectaal bloedverlies|vers bloed per anum|bloeding uit het maag[- ]darmkanaal)\b", re.IGNORECASE)
# # negation_pattern = re.compile(negation_pattern.{0,20}?bleeding_targets, re.IGNORECASE)

# negation_pattern = r"(no|not|without)"  # Example negation triggers
# bleeding_targets = r"(bleeding|rectal)"  # Example bleeding targets

# # pattern = fr"{negation_pattern}(\W+\w+){{0,20}}?{bleeding_targets}"
# pattern = fr"{negation_pattern}(.*?){bleeding_targets}"
# compiled_pattern = re.compile(pattern, re.IGNORECASE)





negation_pattern = re.compile(r"\b(geen|niet|zonder|uitsluiten|ontkent|ontkend|ontkennen|negatief voor|geen aanwijzing voor|kan uitgesloten worden|geen tekenen van)\b.{0,20}?\b(acute gi\s*bloeding|acute maagdarmbloeding|acute gastro[-\s]?intestinale bloeding| agib |acute bloeding|massaal bloedverlies|shock door bloeding|spoed scopie|hematemesis|melaena met hypotensie|mel[ae]na|hematemesis|hematochez[iy]e|rectaal bloed|rectaal verlies|zwart(e)? ontlasting|bloed in ontlasting|divertikelbloeding|(bloed)?stolsels|gastro[-\s]?intestinale bloeding|hemorragie|bloedverlies|vers bloed per anum|bloedende divertikels?|maagdarmbloeding|invloed van antistolling|hemoglobinedaling|bloedende ulcus|bloedend varices?)\b", re.IGNORECASE)




# negation_pattern = re.compile(r"\b(geen|niet|zonder)\b.{0,20}?\b(bloed(braken)?|hematemesis|melena|maagdarmbloeding|gastro[- ]intestinale bloeding|rectaal bloedverlies|vers bloed per anum|bloeding uit het maag[- ]darmkanaal)\b", re.IGNORECASE)
df_measures['negated'] = df_measures['verslagen_report_content'].apply(lambda x: bool(negation_pattern.search(str(x))))
df_measures['negated_match'] = df_measures['verslagen_report_content'].apply(lambda x: extract_match(str(x), negation_pattern))
df_measures['negated_context'] = df_measures['verslagen_report_content'].apply(lambda x: extract_context(str(x), negation_pattern))

df_measures.head()

,pseudo_id,verslagen_report_start_date,verslagen_report_specialism,verslagen_report_content,has_agib,has_gib,has_agib2,has_gib_symptoms,has_scopie_reference,has_gib_treatment,agib_mention,agib_match,agib_context,gib_mention,gib_match,gib_context,negated,negated_match,negated_context
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 15:06:00,"Maag-, Darm- en Leverziekten","Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",False,True,False,True,True,True,False,None,None,True,melena,spontane conversie naar sinusritme 2016 melena...,True,geen rectaal bloedverlies,na de endoscopieën is er geen rectaal bloedver...
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 09:53:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies obv diver...,False,True,False,True,True,True,False,None,None,True,divertikelbloeding,samenvatting rectaal bloedverlies obv divertik...,True,geen rectaal bloedverlies,op colo pandiverticulose beloop vpk geen recta...
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 14:13:00,"Maag-, Darm- en Leverziekten",COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,False,True,False,True,True,True,False,None,None,True,melena,t kamphuis indicatie bloedverlies rectaal mele...,False,None,None
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 13:48:00,"Maag-, Darm- en Leverziekten",GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,False,True,False,True,True,True,False,None,None,True,melena,verwijzer j t kamphuis indicatie melena en blo...,False,None,None
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 08:47:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies ; eenmali...,False,True,False,True,True,True,False,None,None,True,divertikelbloeding,stabiel inr 2 8 wd divertikelbloeding dd toch ...,True,geen stolsels,gehad vannacht donkerrood bloedverlies gehad g...


In [48]:
df_measures['negated'].value_counts()

False    10042
True      1048
Name: negated, dtype: int64

In [49]:
df_measures['has_agib'].value_counts()
len(df_measures[df_measures['has_agib']]['pseudo_id'].unique())

3

In [50]:
for i, row in df_measures[df_measures['has_agib'] == True].iterrows():
    print(f"🩸 Date: {row['verslagen_report_start_date']}")
    print(f"🩸 Match: {row['has_agib']}")
    print(f"🩸 AGIB metnion: {row['agib_mention']}")
    print(f"🩸 AGIB match: {row['agib_match']}")
    print(f"🩸 AGIB context: {row['agib_context']}")
    # print(f"📍 Context: {row['agib_context']}")
    print(f"📜 Content: {row['verslagen_report_content']}")
    print("-----")

🩸 Date: 2021-06-24 15:27:00
🩸 Match: True
🩸 AGIB metnion: False
🩸 AGIB match: None
🩸 AGIB context: None
📜 Content: Samenvatting: 
Acute bloeding in dunne darm bij VCE onder clopidogrel. Gastro-jejunoscopie + ileocoloscopie: gb. DBE tot 200cm voorbij Treitz in ETZ onder afwijkingen.
MRI: afwijkingen onderbuik; massa rechter adnex wv icc gyn loopt.
VG hemolytische anemie, mamma ca, TIA, clopidogrel (tijdelijk gestopt).


Beloop: 
onderzoek gaat niet door, pte belt dit door.
-----
🩸 Date: 2021-06-03 11:31:00
🩸 Match: True
🩸 AGIB metnion: False
🩸 AGIB match: None
🩸 AGIB context: None
📜 Content: Samenvatting: 
Acute bloeding in dunne darm bij VCE onder clopidogrel. Gastro-jejunoscopie + ileocoloscopie: gb. DBE tot 200cm voorbij Treitz in ETZ onder afwijkingen.
MRI: afwijkingen onderbuik; massa rechter adnex wv icc gyn loopt.
VG hemolytische anemie, mamma ca, TIA, clopidogrel (tijdelijk gestopt).


Beloop: 
Telefonisch i.v.m. COVID:
Voelt zich onzeker. 
Geen bloedverlies meer gehad. 
Zwak we